# Sample program

## 1. Install required packages

In [2]:
! pip install -q transformers==4.41.2
! pip install -q bitsandbytes==0.43.1
! pip install -q accelerate==0.31.0
! pip install -q langchain==0.2.5
! pip install -q langchainhub==0.1.20
! pip install -q langchain-chroma==0.1.1
! pip install -q langchain-community==0.2.5
! pip install -q langchain_huggingface==0.0.3
! pip install -q python-dotenv==1.0.1
! pip install -q pypdf==4.2.0
! pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

## 2. Build the vector database

### 2.1 Import packages

In [3]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

### 2.2 Load the PDF file

In [9]:
! curl -L "https://drive.usercontent.google.com/download?id=1lWuq0COKnU9mCfMvTEq54DBLgAh3yYDx&export=download&confirm=t" -o sample.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.8M  100 15.8M    0     0  8722k      0  0:00:01  0:00:01 --:--:-- 8722k


In [12]:
Loader = PyPDFLoader
FILE_PATH = "sample.pdf"
loader = Loader(FILE_PATH)
documents = loader.load()

### 2.3 Initialize the text splitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap=100)

In [15]:
docs = text_splitter.split_documents ( documents )

print("Number of chunks:", len(docs))
print(docs[0])

Number of chunks: 33
page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI. Giới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look\nOnce) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực\nthi mà loại mô hình này mang lại.\nHình 1: Logo của mô hình YOLO. Ảnh: link.\nThời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)\nđã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu 

### 2.4 Initialize the vectorization (embedding) instance

In [16]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
embedding.embed_query("What is YOLOv9?")

[0.0843772143125534,
 -0.0203261636197567,
 0.026446180418133736,
 0.035629753023386,
 -0.00866309367120266,
 0.012607274577021599,
 0.07114778459072113,
 -0.0008519569528289139,
 -0.022037317976355553,
 0.014154433272778988,
 0.027396107092499733,
 -0.023776333779096603,
 0.029847750440239906,
 0.02023252658545971,
 0.01728552021086216,
 0.016114424914121628,
 -0.004818771965801716,
 0.009413986466825008,
 -0.01709321141242981,
 -0.016088522970676422,
 -0.003163769841194153,
 0.004725933540612459,
 -0.025693301111459732,
 0.04460318014025688,
 -0.0217585451900959,
 -0.02698495052754879,
 0.00532176299020648,
 0.0315939225256443,
 -0.04906630516052246,
 -0.016287555918097496,
 0.007594667840749025,
 -0.022139057517051697,
 -0.046068642288446426,
 0.0746874213218689,
 1.609064838703489e-06,
 0.003023634199053049,
 -0.02150876261293888,
 0.012858264148235321,
 0.03915918618440628,
 -0.036399926990270615,
 -0.05311734229326248,
 0.0217827707529068,
 -0.011551699601113796,
 -0.006327142007

### 2.5 Initialize the vector database

In [17]:
vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vector_db.as_retriever()

In [37]:
# Demonstration. Adjust the input according to the actual content of the document
result = retriever.invoke("What is YOLOv9?")
print("Number of relevant documents:", len(result))
# print(result[0].page_content)
# print(result[0].page_content in [doc.page_content for doc in docs])
for res in result:
  print('-'*30)
  print(res.page_content)


Number of relevant documents: 4
------------------------------
AI VIETNAM (AIO2024) aivietnam.edu.vn
1! mkdir datasets
2! unzip -q "/ content / PlantDocv4 . zip" -d "/ content / datasets / PlantDocv4
/"
3!rm / content / PlantDocv4 .zip
Quan sát thư mục giải nén, có thể thấy bộ dữ liệu này đã được gán nhãn và đưa vào format
cấu trúc dữ liệu training theo yêu cầu của YOLO. Vì vậy, chúng ta sẽ không cần thực hiện
bước chuẩn bị dữ liệu ở bài này.
2.Cài đặt và import các thư viện cần thiết: Tương tự như phần trước, các bạn chạy
các đoạn code sau để cài đặt các gói thư viện để sử dụng được YOLOv10:
1!git clone https :// github .com/THU -MIG/ yolov10 .git
2%cd yolov10
3!pip install -q -r requirements .txt
4!pip install -e .
3.Khởi tạo mô hình YOLOv10: Chúng ta sẽ khởi tạo mộ hình YOLOv10 với phiên
bảnnano (n) từ trọng số đã được huấn luyện trên bộ dữ liệu COCO. Để tải trọng số
yolov10n.pt, các bạn chạy đoạn code sau:
1! wget https :// github .com/THU -MIG/ yolov10 / releases / download /v1 .1

## 3. Initialize the Vicuna Large Language Model

### 3.1 Define configuaration of the model

In [42]:
nf4_config = BitsAndBytesConfig (load_in_4bit=True ,
                                 bnb_4bit_quant_type="nf4",
                                 bnb_4bit_use_double_quant=True ,
                                 bnb_4bit_compute_dtype=torch.bfloat16)

### 3.2 Initialize the model and the tokenizer

In [44]:
MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                             quantization_config=nf4_config,
                                             low_cpu_mem_usage = True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

pytorch_model-00001-of-00002.bin:   6%|5         | 598M/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

### 3.3 Integrate the model, the tokenizer into a pipeline

In [46]:
model_pipeline = pipeline("text-generation",
                          model=model,
                          tokenizer = tokenizer,
                          max_new_tokens =512,
                          pad_token_id=tokenizer.eos_token_id,
                          device_map ="auto")

llm = HuggingFacePipeline(pipeline=model_pipeline)

## 4. Run the program

In [56]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


YOLOv10 là một phiên bản của YOLO (You Only Look Once) - một hệ thống dự đoán hình ảnh được huấn luyện sẵn trên bộ dữ liệu COCO. Phiên bản này được tạo bằng cách tải về trọng số (file.pt) từ GitHub và khởi tạo mô hình bằng cách sử dụng thư viện ultralytics.


In [60]:
print(type(rag_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [59]:
type({} | prompt)

langchain_core.runnables.base.RunnableSequence

# Build chat application with Chainlit

## 1. Install packages

In [67]:
! pip install -q langchain-openai==0.1.9
! pip install -q chainlit==1.1.304
! npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.784s


## 2. Import packages

In [68]:
import chainlit as cl
import torch

from chainlit.types import AskFileResponse

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader , TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

## 3. Define function for processing input file

In [65]:
def process_file(file: AskFileResponse):
  if file.type == "text/plain":
    Loader = TextLoader
  elif file . type == "application/pdf":
    Loader = PyPDFLoader

  loader = Loader(file.path)
  document = loader.load()
  docs = text_splitter.split_documents(document)
  for i, doc in enumerate(docs):
    doc.metadata["source"] = f"source_{i}"

  return docs

## 4. Define text splitter and tokenizer

In [70]:
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=100)
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 4. Initialize the vector database

In [71]:
def get_vector_db (file: AskFileResponse ):
  docs = process_file(file)
  cl.user_session.set("docs", docs)
  vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
  return vector_db

## 5. Initialize the Vicuna Large Language Model

In [72]:
def get_huggingface_llm(model_name: str="lmsys/vicuna-7b-v1.5", max_new_token: int=512) :
  nf4_config = BitsAndBytesConfig (load_in_4bit=True ,
                                 bnb_4bit_quant_type="nf4",
                                 bnb_4bit_use_double_quant=True ,
                                 bnb_4bit_compute_dtype=torch.bfloat16)

  model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                              quantization_config=nf4_config,
                                              low_cpu_mem_usage = True)
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  model_pipeline = pipeline("text-generation",
                          model=model,
                          tokenizer = tokenizer,
                          max_new_tokens =512,
                          pad_token_id=tokenizer.eos_token_id,
                          device_map ="auto")

  llm = HuggingFacePipeline(pipeline=model_pipeline)
  return llm

LLM = get_huggingface_llm()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

## 7. Define welcome message

In [ ]:
welcome_message = """Welcome to the PDF QA! To get started :
1. Upload a PDF or text file
2. Ask a question about the file
"""

## 8. Define on_chat_start async function

In [73]:
@cl.on_chat_start
async def on_chat_start():
  files = None
  while files is None :
    files = await cl.AskFileMessage(content=welcome_message,
                                     accept=["text/plain", "application/pdf"],
                                     max_size_mb=20 ,
                                     timeout=180).send()
  file = files[0]
  msg = cl.Message(content=f"Processing '{file.name}'...",
                   disable_feedback=True)
  await msg.send()

  vector_db = await cl.make_async(get_vector_db)(file)

  message_history = ChatMessageHistory()
  memory = ConversationBufferMemory(memory_key="chat_history",
                                    output_key ="answer",
                                    chat_memory=message_history,
                                    return_messages =True)
  retriever = vector_db.as_retriever(search_type="mmr",
                                     search_kwargs={'k': 3})

  chain = ConversationalRetrievalChain.from_llm (llm =LLM,
                                                 chain_type="stuff",
                                                 retriever=retriever,
                                                 memory=memory,
                                                 return_source_documents=True)

  msg.content = f"'{file.name}' processed. You can now ask questions!"
  await msg.update()

  cl.user_session.set("chain", chain)

## 9. Define on_message async function

In [74]:
@cl . on_message
async def on_message(message: cl.Message):
  chain = cl.user_session.get("chain")
  cb = cl.AsyncLangchainCallbackHandler()
  res = await chain.ainvoke(message.content, callbacks =[cb])
  answer = res["answer"]
  source_documents = res["source_documents"]
  text_elements = []

  if source_documents :
    for source_idx , source_doc in enumerate(source_documents):
      source_name = f"source_{source_idx}"
      text_elements.append(cl.Text(content=source_doc.page_content,
                                    name=source_name))
      source_names = [text_el.name for text_el in text_elements]

      if source_names :
        answer += f"\ nSources : {', '.join(source_names)}"
      else:
        answer += "\nNo sources found"

  await cl.Message(content=answer, elements=text_elements).send()

## 10. Run Chainlit app

In [76]:
! chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt &

## 11. Expose local host using localtunnel (currently unavailable)

In [78]:
import urllib

In [84]:
print("Password/Enpoint IP for localtunnel is:",
      urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.204.5.97


In [85]:
!lt --port 8000 -- subdomain aivn-simple-rag

your url is: https://itchy-hats-flash.loca.lt
^C
